In [1]:
import pandas as pd
import numpy as np
from model import get_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import ReLU
from tensorflow.keras.activations import tanh
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
import keras

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 3)                 9         
                                                                 
 dense_1 (Dense)             (None, 2)                 8         
                                                                 
 dense_2 (Dense)             (None, 3)                 9         
                                                                 
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________________
None
[<tf.Variable 'dense/kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.8708074 , -0.87618303, -0.13699722],
       [ 0.05719924,  0.21344233, -0.7213781 ],
       [ 0.4240322 , -0.476822

In [2]:
rain = pd.read_csv("./Data/Rainfall.csv")

In [3]:
import glob

slp_path = "./Data/slp/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)
  
slp_months.sort()
print(slp_months)

['./Data/slp\\01_slp.csv', './Data/slp\\02_slp.csv', './Data/slp\\03_slp.csv', './Data/slp\\04_slp.csv', './Data/slp\\05_slp.csv', './Data/slp\\06_slp.csv', './Data/slp\\07_slp.csv', './Data/slp\\08_slp.csv', './Data/slp\\09_slp.csv', './Data/slp\\10_slp.csv', './Data/slp\\11_slp.csv', './Data/slp\\12_slp.csv']


In [4]:
#Here i am just storing rainfall data of Jun to sep from 1949 to 2014
# index 47= 1948,
import statistics
lpa = statistics.mean(list(rain['Jun-Sep'][60:89]))
print(lpa)
rain_fall_data = list(rain['Jun-Sep'][47:2020])
rain_data = []
for i in range(len(rain_fall_data)):
  temp = (rain_fall_data[i]/lpa)*100
  rain_data.append(temp)

878.0689655172414


In [5]:
from scipy.stats import pearsonr
correlation = []

# chacking for each MONTH
month_path = slp_months[1] #0th in dex means january
# reading month data
month_data = pd.read_csv(month_path)
# month_data.head()
# len(month_data)

In [6]:
month_data.iloc[0]
# To extract only the relavant infromation from data frame for autoencoder
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  # print(new_data)
  return np.array(new_data)

combine = []
for i in range(12):
  month_path = slp_months[i]
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  feature = feature[0:53]
  for i in feature:
    combine.append(i)
x = np.array(combine)
x.shape

(636, 324)

In [19]:
def get_lr(initial_learning_rate=0.001, decay_steps=4000, decay_rate=1):
    return optimizers.schedules.ExponentialDecay(
            initial_learning_rate,
            decay_steps=decay_steps,
            decay_rate=decay_rate,
            staircase=True)

In [20]:
# define encoder
def get_model_1(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  # e = Dense(n_inputs, use_bias=False)(visible)
  e = ReLU()(visible)
  bottleneck = Dense(97, use_bias=False)(e)
  output = Dense(n_inputs, activation="linear")(bottleneck)

  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer=optimizers.Adagrad(learning_rate=get_lr()), loss='mse')
  return model

In [21]:
model_1 = get_model_1()
model_1.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 324)]             0         
                                                                 
 re_lu_3 (ReLU)              (None, 324)               0         
                                                                 
 dense_9 (Dense)             (None, 97)                31428     
                                                                 
 dense_10 (Dense)            (None, 324)               31752     
                                                                 
Total params: 63,180
Trainable params: 63,180
Non-trainable params: 0
_________________________________________________________________


In [22]:
history = model_1.fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))

Epoch 1/800
40/40 - 0s - loss: 16608.4609 - val_loss: 14398.0791 - 273ms/epoch - 7ms/step
Epoch 2/800
40/40 - 0s - loss: 13413.7959 - val_loss: 12445.7881 - 57ms/epoch - 1ms/step
Epoch 3/800
40/40 - 0s - loss: 11871.4814 - val_loss: 11273.9199 - 91ms/epoch - 2ms/step
Epoch 4/800
40/40 - 0s - loss: 10908.8008 - val_loss: 10503.3936 - 68ms/epoch - 2ms/step
Epoch 5/800
40/40 - 0s - loss: 10250.7012 - val_loss: 9955.2637 - 61ms/epoch - 2ms/step
Epoch 6/800
40/40 - 0s - loss: 9767.7822 - val_loss: 9533.2920 - 70ms/epoch - 2ms/step
Epoch 7/800
40/40 - 0s - loss: 9388.3027 - val_loss: 9199.9375 - 57ms/epoch - 1ms/step
Epoch 8/800
40/40 - 0s - loss: 9083.8242 - val_loss: 8926.2441 - 58ms/epoch - 1ms/step
Epoch 9/800
40/40 - 0s - loss: 8832.4551 - val_loss: 8696.3730 - 55ms/epoch - 1ms/step
Epoch 10/800
40/40 - 0s - loss: 8618.0020 - val_loss: 8501.0840 - 57ms/epoch - 1ms/step
Epoch 11/800
40/40 - 0s - loss: 8434.8398 - val_loss: 8331.4453 - 55ms/epoch - 1ms/step
Epoch 12/800
40/40 - 0s - loss:

In [11]:
model_1.layers[3].get_weights()[0].shape

(97, 324)

In [12]:
model_temp1 = model_final1 = Model(inputs=model_1.input, outputs=model_1.layers[2].output)
model_2_inputs = model_temp1(x)
model_2_inputs.shape

TensorShape([636, 97])

In [13]:
# define encoder
def get_model_2(n_inputs=97):
  visible = Input(shape=(n_inputs,))

  # e = Dense(n_inputs, use_bias=False)(visible)
  e = ReLU()(visible)
  bottleneck = Dense(29, use_bias=False)(e)
  output = Dense(n_inputs)(bottleneck)

  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer=optimizers.Adagrad(learning_rate=get_lr(0.07, 2000, 1.1)), loss='mse')
  return model

In [14]:
model_2 = get_model_2()
model_2.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 97)]              0         
                                                                 
 re_lu_1 (ReLU)              (None, 97)                0         
                                                                 
 dense_5 (Dense)             (None, 29)                2813      
                                                                 
 dense_6 (Dense)             (None, 97)                2910      
                                                                 
Total params: 5,723
Trainable params: 5,723
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model_2.fit(model_2_inputs, model_2_inputs, epochs=800, batch_size=16, verbose=2, validation_data=(model_2_inputs,model_2_inputs))

NameError: name 'model_2' is not defined

In [ ]:
model_temp2 = model_final1 = Model(inputs=model_2.input, outputs=model_2.layers[3].output)
model_3_inputs = model_temp2(model_2_inputs)
model_3_inputs.shape

TensorShape([636, 29])

In [ ]:
def get_model_3(n_inputs=29):
  visible = Input(shape=(n_inputs,))

  e = Dense(n_inputs, use_bias=False)(visible)
  e = ReLU()(e)
  bottleneck = Dense(9)(e)
  output = Dense(n_inputs, use_bias=False)(bottleneck)

  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer=optimizers.Adagrad(learning_rate=get_lr(0.1, 2000, 1.1)), loss='mse')
  return model

In [ ]:
model_3 = get_model_3()
model_3.summary()

Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 29)]              0         
                                                                 
 dense_47 (Dense)            (None, 29)                841       
                                                                 
 re_lu_21 (ReLU)             (None, 29)                0         
                                                                 
 dense_48 (Dense)            (None, 9)                 270       
                                                                 
 dense_49 (Dense)            (None, 29)                261       
                                                                 
Total params: 1,372
Trainable params: 1,372
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# fit the autoencoder model to reconstruct input
history = model_3.fit(model_3_inputs, model_3_inputs, epochs=1000, batch_size=16, verbose=2, validation_data=(model_3_inputs,model_3_inputs))
# plot loss

Epoch 1/1000
40/40 - 0s - loss: 14347.0078 - val_loss: 10865.5576 - 274ms/epoch - 7ms/step
Epoch 2/1000
40/40 - 0s - loss: 10642.4297 - val_loss: 9949.5938 - 49ms/epoch - 1ms/step
Epoch 3/1000
40/40 - 0s - loss: 9969.7002 - val_loss: 9533.0869 - 52ms/epoch - 1ms/step
Epoch 4/1000
40/40 - 0s - loss: 9594.7500 - val_loss: 9288.6660 - 51ms/epoch - 1ms/step
Epoch 5/1000
40/40 - 0s - loss: 9369.2402 - val_loss: 9126.7969 - 51ms/epoch - 1ms/step
Epoch 6/1000
40/40 - 0s - loss: 9218.5840 - val_loss: 8997.7520 - 50ms/epoch - 1ms/step
Epoch 7/1000
40/40 - 0s - loss: 9093.7109 - val_loss: 8894.4404 - 53ms/epoch - 1ms/step
Epoch 8/1000
40/40 - 0s - loss: 9003.6953 - val_loss: 8841.4258 - 51ms/epoch - 1ms/step
Epoch 9/1000
40/40 - 0s - loss: 8936.0752 - val_loss: 8768.9863 - 48ms/epoch - 1ms/step
Epoch 10/1000
40/40 - 0s - loss: 8867.1631 - val_loss: 8725.9980 - 52ms/epoch - 1ms/step
Epoch 11/1000
40/40 - 0s - loss: 8825.4512 - val_loss: 8693.3408 - 48ms/epoch - 1ms/step
Epoch 12/1000
40/40 - 0s -

In [ ]:
def get_model_final(n_inputs=324):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs, use_bias=False)(visible)
  e = ReLU()(e)
  e = Dense(97, use_bias=False)(e)
  e = ReLU()(e)
  e = Dense(29, use_bias=False)(e)
  e = ReLU()(e)
  bottleneck = Dense(9)(e)
  e = Dense(29)(bottleneck)
  e = ReLU()(e)
  e = Dense(97)(e)
  e = ReLU()(e)
  output = Dense(n_inputs, activation='linear')(e)

  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [ ]:
model_final = get_model_final()
model_final.summary()

Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 324)]             0         
                                                                 
 dense_50 (Dense)            (None, 324)               104976    
                                                                 
 re_lu_22 (ReLU)             (None, 324)               0         
                                                                 
 dense_51 (Dense)            (None, 97)                31428     
                                                                 
 re_lu_23 (ReLU)             (None, 97)                0         
                                                                 
 dense_52 (Dense)            (None, 29)                2813      
                                                                 
 re_lu_24 (ReLU)             (None, 29)                0  

In [ ]:
model_final.layers[1] = model_1.layers[1]
model_final.layers[2] = model_1.layers[2]
model_final.layers[3] = model_2.layers[1]
model_final.layers[4] = model_2.layers[2]
model_final.layers[5] = model_3.layers[1]
model_final.layers[6] = model_3.layers[2]
model_final.layers[7] = model_3.layers[3]
model_final.layers[8] = model_3.layers[4]
model_final.layers[10] = model_2.layers[4]
model_final.layers[12] = model_1.layers[4]

In [ ]:
history = model_final.fit(x, x, epochs=1000, batch_size=16, verbose=2, validation_data=(x,x))


Epoch 1/1000
40/40 - 0s - loss: 12955.3828 - val_loss: 9395.7939 - 420ms/epoch - 10ms/step
Epoch 2/1000
40/40 - 0s - loss: 8366.7158 - val_loss: 7832.4976 - 73ms/epoch - 2ms/step
Epoch 3/1000
40/40 - 0s - loss: 7707.0732 - val_loss: 7403.7910 - 71ms/epoch - 2ms/step
Epoch 4/1000
40/40 - 0s - loss: 7278.0723 - val_loss: 6937.4307 - 72ms/epoch - 2ms/step
Epoch 5/1000
40/40 - 0s - loss: 6809.4731 - val_loss: 6524.2734 - 74ms/epoch - 2ms/step
Epoch 6/1000
40/40 - 0s - loss: 6425.6118 - val_loss: 6150.0210 - 72ms/epoch - 2ms/step
Epoch 7/1000
40/40 - 0s - loss: 6080.3540 - val_loss: 5889.4673 - 75ms/epoch - 2ms/step
Epoch 8/1000
40/40 - 0s - loss: 5841.8940 - val_loss: 5647.6802 - 77ms/epoch - 2ms/step
Epoch 9/1000
40/40 - 0s - loss: 5668.4131 - val_loss: 5485.8633 - 76ms/epoch - 2ms/step
Epoch 10/1000
40/40 - 0s - loss: 5453.4014 - val_loss: 5289.9971 - 76ms/epoch - 2ms/step
Epoch 11/1000
40/40 - 0s - loss: 5278.5845 - val_loss: 5154.4434 - 74ms/epoch - 2ms/step
Epoch 12/1000
40/40 - 0s - 

In [ ]:
model_final1 = Model(inputs=model_final.input, outputs=model_final.layers[3].output)
model_final1.trainable = False

model_final2 = Model(inputs=model_final.input, outputs=model_final.layers[5].output)
model_final2.trainable = False

model_final3 = Model(inputs=model_final.input, outputs=model_final.layers[7].output)
model_final3.trainable = False

models = [model_final1, model_final2, model_final3]
# models = [model_final1, model_final4]

In [ ]:
month_data.iloc[0]
# To extract only the relavant infromation from data frame for autoencoder
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  # print(new_data)
  return np.array(new_data)

# to combine all the features together
def predictor(month,mod):
  # chacking for each MONTH
  month_path = slp_months[month] #0th in dex means january
  # reading month data
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  feature = feature[0:53]  #Here i am using anly data from 1948 to 2000
  pred = np.array(feature)
  pred_m = model_final1(x)
  pred_f = pred_m.numpy()
  return pred_f

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


#w_start = window start, w_end = window end  
def get_top_pred(top,pred_info,w_start,w_end):
  k = 0
  pred_f = pred_info
  cor_list = []
  for i in range(len(pred_f[0])):
    score = []
    
    for j in range(w_start,w_end):
      score.append(pred_f[j][i])
    corr, _ = pearsonr(rain_fall_data[w_start:w_end],score)
    cor_list.append(corr)
    # print(corr)
  list1=list(enumerate(cor_list)) #I have taken cosine rank[0] beacuse it is inside the list
  list2=sorted(list1, key=lambda x: x[1],reverse=True)
  top_feature_index = []
  #here i am getting the top features index
  for i in range(top):
    index = list2[i][0]
    top_feature_index.append(index)
  # Here i am getting all the top features
  predictor = []
  for i in range(len(pred_f)):
    temp = []
    for j in top_feature_index:
      # print(i,"  ",j)
      feature = pred_f[i][j]
      temp.append(feature)
    predictor.append(temp)
  # print(list2)
  return predictor
  # print(list1)


# np.concatenate((a, b.T), axis=1)
def get_predictors(months,top,models,w_start,w_end):
  for i in months:
    for mod in range(4):
      y_pred = predictor(i,mod)
      b = get_top_pred(top, y_pred, w_start,w_end)
      try:    pred = np.concatenate((pred,b), axis=1)
      except: pred = np.array(b)
  return pred

def window_solution(months,top):
  reg = SVR(kernel = 'rbf',C=1.0,epsilon=0.45)
  # reg = make_pipeline(StandardScaler(), SVR(kernel = 'rbf',C=1.0, epsilon=0.2))
  # months = [0]
  k=0
  cor_all = []
  for window in range(10,53):
    k+=1
    score = []
    pred = get_predictors(months,top,models,53-window,53)
    for i in range(14):
      start = 53-window
      end = 53+1
      # pred = get_predictors(months,2,models,start,end)
      reg.fit(pred[53-window:53+i], rain_fall_data[53-window:53+i])
      score.append(reg.predict([pred[53+i]])[0])
    # print(score)
    corr, _ = pearsonr(rain_fall_data[53:67],score)
    print("Window size = ",window,"   plcc",corr)
    cor_all.append(corr)
  return min(cor_all),max(cor_all)
#This is by taking correlation after every iteraion of each predict

In [ ]:
months_comb = [[4]]
top = 9
models = [model_final2 ]
for month in months_comb:
  for i in range(1,top):
    left, right = window_solution(month,i)
    print(" Month combination = ",month,"    top = ",i,"   min and max",left,"   ",right)

Window size =  10    plcc -0.21417038804216296
Window size =  11    plcc -0.3264243271447258
Window size =  12    plcc -0.4488066376556242
Window size =  13    plcc -0.39388960466461664
Window size =  14    plcc -0.4317306566504997
Window size =  15    plcc -0.3132616702989745
Window size =  16    plcc -0.06404799696546132
Window size =  17    plcc -0.2444313399858756
Window size =  18    plcc -0.3341367886175244
Window size =  19    plcc -0.4602775915385764
Window size =  20    plcc -0.4646785068019082
Window size =  21    plcc -0.21605604428086395
Window size =  22    plcc -0.4882143158304113
Window size =  23    plcc -0.28812378609069955
Window size =  24    plcc -0.2060523374916734
Window size =  25    plcc -0.2945884238265506
Window size =  26    plcc -0.2716370034213569
Window size =  27    plcc -0.30354290888127067
Window size =  28    plcc -0.24351163899464942
Window size =  29    plcc -0.3008746494268277
Window size =  30    plcc -0.28014884204362756
Window size =  31    plcc 